# 0. Preparación del ambiente

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = ''
os.environ["AWS_SECRET_ACCESS_KEY"] = ''
os.environ["AWS_SESSION_TOKEN"] = ''

# 1. Manejo de sesiones

### S3

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("covid19_colombia")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.amazonaws:aws-java-sdk:1.11.1034,org.apache.hadoop:hadoop-aws:3.3.4")\
    .config('fs.s3a.access.key', os.environ["AWS_ACCESS_KEY_ID"]) \
    .config('fs.s3a.secret.key', os.environ["AWS_SECRET_ACCESS_KEY"]) \
    .config('fs.s3a.session.token', os.environ["AWS_SESSION_TOKEN"]) \
    .config('fs.s3a.path.style.access', 'true') \
    .config('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config('fs.s3a.endpoint', 's3.amazonaws.com') \
    .getOrCreate()

sc = spark.sparkContext
csv_path = "s3a://msosavp3/covid19_colombia.csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)

### Google Drive

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("covid19_colombia")\
    .master("local[*]")\
    .getOrCreate()

sc = spark.sparkContext
csv_path = "/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/covid19_colombia.csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)

# 2. Análisis exploratorio

## Mostrar el DataFrame

In [ ]:
df.show()

+-------------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+---------+-------------------+---------------+----------+---------------------------+-------------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|  fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|   Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|    Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-------------------+----------+---------------------+----------------------------+-------------------+-------------------------+-----------

## 2.1 y 2.2 Mostrar las columnas y el tipo de dato de estas

In [ ]:
df.printSchema()

root
 |-- fecha reporte web: timestamp (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: timestamp (nullable = true)
 |-- Fecha de muerte: timestamp (nullable = true)
 |-- Fecha de diagnóstico: timestamp (nullable = true)
 |-- Fecha de recuperación: timestamp (nullable = tr

## 2.3 Seleccionar algunas columnas

In [ ]:
selected_columns = df.select("ID de caso", "Nombre departamento", "Nombre municipio", "Edad", "Sexo", "Estado")
selected_columns.show(5)

+----------+-------------------+----------------+----+----+---------+
|ID de caso|Nombre departamento|Nombre municipio|Edad|Sexo|   Estado|
+----------+-------------------+----------------+----+----+---------+
|   1556979|              VALLE|            CALI|  67|   F|     Leve|
|   1556980|              VALLE|            CALI|  66|   F|     Leve|
|   1556981|              VALLE|            CALI|  68|   F|     Leve|
|   1556982|              VALLE|            CALI|  74|   F|Fallecido|
|   1556983|              VALLE|            CALI|  65|   F|     Leve|
+----------+-------------------+----------------+----+----+---------+
only showing top 5 rows



## 2.4 Renombrar las columnas

In [ ]:
df = df.withColumnRenamed("fecha reporte web", "fecha_reporte") \
      .withColumnRenamed("ID de caso", "id") \
      .withColumnRenamed("Fecha de notificación", "fecha_notificacion") \
      .withColumnRenamed("Fecha de inicio de síntomas", "fecha_sintomas") \
      .withColumnRenamed("Fecha de muerte", "fecha_muerte") \
      .withColumnRenamed("Fecha de diagnóstico", "fecha_diagnostico") \
      .withColumnRenamed("Fecha de recuperación", "fecha_recuperacion") \
      .withColumnRenamed("Recuperado", "recuperado") \
      .withColumnRenamed("Estado", "estado") \
      .withColumnRenamed("Edad", "edad") \
      .withColumnRenamed("Nombre departamento", "departamento") \
      .withColumnRenamed("Nombre municipio", "municipio") \
      .withColumnRenamed("Unidad de medida de edad", "unidad_edad") \
      .withColumnRenamed("Tipo de contagio", "tipo_contagio") \
      .withColumnRenamed("Ubicación del caso", "ubicacion") \
      .withColumnRenamed("Código DIVIPOLA departamento", "divipola_departamento") \
      .withColumnRenamed("Código DIVIPOLA municipio", "divipola_municipio") \
      .withColumnRenamed("Código ISO del país", "iso") \
      .withColumnRenamed("Nombre del país", "pais") \
      .withColumnRenamed("Tipo de recuperación", "tipo_recuperacion") \
      .withColumnRenamed("Pertenencia étnica", "pertenencia_etnica") \
      .withColumnRenamed("Nombre del grupo étnico", "grupo_etnico")

df.printSchema()

root
 |-- fecha_reporte: timestamp (nullable = true)
 |-- id: integer (nullable = true)
 |-- fecha_notificacion: timestamp (nullable = true)
 |-- divipola_departamento: integer (nullable = true)
 |-- departamento: string (nullable = true)
 |-- divipola_municipio: integer (nullable = true)
 |-- municipio: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- unidad_edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- tipo_contagio: string (nullable = true)
 |-- ubicacion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- iso: integer (nullable = true)
 |-- pais: string (nullable = true)
 |-- recuperado: string (nullable = true)
 |-- fecha_sintomas: timestamp (nullable = true)
 |-- fecha_muerte: timestamp (nullable = true)
 |-- fecha_diagnostico: timestamp (nullable = true)
 |-- fecha_recuperacion: timestamp (nullable = true)
 |-- tipo_recuperacion: string (nullable = true)
 |-- pertenencia_etnica: integer (nullable = true)
 |-- grupo_et

## 2.5 Agregar columnas

In [ ]:
from pyspark.sql.functions import col, datediff

df = df.withColumn("fecha_sintomas", col("fecha_sintomas").cast("timestamp"))
df = df.withColumn("fecha_recuperacion", col("fecha_recuperacion").cast("timestamp"))

df = df.withColumn("dias_contagiado",
                   datediff(col("fecha_recuperacion"), col("fecha_sintomas")))

df.select("fecha_sintomas", "fecha_recuperacion", "dias_contagiado").show(5)


+-------------------+-------------------+---------------+
|     fecha_sintomas| fecha_recuperacion|dias_contagiado|
+-------------------+-------------------+---------------+
|2020-12-21 00:00:00|2021-01-04 00:00:00|             14|
|2020-12-07 00:00:00|2020-12-25 00:00:00|             18|
|2020-12-18 00:00:00|2021-01-01 00:00:00|             14|
|2020-12-17 00:00:00|               NULL|           NULL|
|2020-12-21 00:00:00|2021-01-04 00:00:00|             14|
+-------------------+-------------------+---------------+
only showing top 5 rows



## 2.6 Borrar columnas

In [ ]:
df = df.drop("iso")
df.show(5)

+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------+---------------+
|      fecha_reporte|     id| fecha_notificacion|divipola_departamento|departamento|divipola_municipio|municipio|edad|unidad_edad|Sexo|tipo_contagio|ubicacion|   estado|pais|recuperado|     fecha_sintomas|       fecha_muerte|  fecha_diagnostico| fecha_recuperacion|tipo_recuperacion|pertenencia_etnica|grupo_etnico|dias_contagiado|
+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------+---------------+
|202

## 2.7 Filtrar datos

In [ ]:
filtered_df = df.filter(df["estado"] == "Fallecido")
filtered_df.show(5)

+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+------------------+-----------------+------------------+------------+---------------+
|      fecha_reporte|     id| fecha_notificacion|divipola_departamento|departamento|divipola_municipio|municipio|edad|unidad_edad|Sexo|tipo_contagio|ubicacion|   estado|pais|recuperado|     fecha_sintomas|       fecha_muerte|  fecha_diagnostico|fecha_recuperacion|tipo_recuperacion|pertenencia_etnica|grupo_etnico|dias_contagiado|
+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+------------------+-----------------+------------------+------------+---------------+
|2020-1

## 2.8 Función UDF

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def edad_grupo(edad):
    if edad < 18:
        return "Menor"
    elif 18 <= edad < 60:
        return "Adulto"
    else:
        return "Mayor"

edad_grupo_udf = udf(edad_grupo, StringType())

df = df.withColumn("grupo_edad", edad_grupo_udf(df["edad"]))
df.show(5)

+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------+---------------+----------+
|      fecha_reporte|     id| fecha_notificacion|divipola_departamento|departamento|divipola_municipio|municipio|edad|unidad_edad|Sexo|tipo_contagio|ubicacion|   estado|pais|recuperado|     fecha_sintomas|       fecha_muerte|  fecha_diagnostico| fecha_recuperacion|tipo_recuperacion|pertenencia_etnica|grupo_etnico|dias_contagiado|grupo_edad|
+-------------------+-------+-------------------+---------------------+------------+------------------+---------+----+-----------+----+-------------+---------+---------+----+----------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------

# 3. Preguntas de negocio

In [ ]:
df.createOrReplaceTempView("covid_data")

## 3.1 Los 10 departamentos con más casos de COVID en Colombia

In [ ]:
from pyspark.sql.functions import desc

departamentos_con_mas_casos = df.groupBy("departamento").count().orderBy(desc("count")).limit(10)
departamentos_con_mas_casos.show()

+------------+-------+
|departamento|  count|
+------------+-------+
|      BOGOTA|1888137|
|   ANTIOQUIA| 955271|
|       VALLE| 572724|
|CUNDINAMARCA| 331331|
|   SANTANDER| 297370|
|BARRANQUILLA| 277989|
|   CARTAGENA| 163526|
|   ATLANTICO| 141072|
|      BOYACA| 131133|
|      TOLIMA| 127764|
+------------+-------+



In [ ]:
spark.sql("""
          SELECT `departamento` AS departamento, COUNT(*) AS casos_totales
          FROM covid_data
          GROUP BY `departamento`
          ORDER BY casos_totales DESC
          LIMIT 10
          """).show()


+------------+-------------+
|departamento|casos_totales|
+------------+-------------+
|      BOGOTA|      1888137|
|   ANTIOQUIA|       955271|
|       VALLE|       572724|
|CUNDINAMARCA|       331331|
|   SANTANDER|       297370|
|BARRANQUILLA|       277989|
|   CARTAGENA|       163526|
|   ATLANTICO|       141072|
|      BOYACA|       131133|
|      TOLIMA|       127764|
+------------+-------------+



## 3.2 Las 10 ciudades con mas casos de COVID en Colombia

In [ ]:
municipios_con_mas_casos = df.groupBy("municipio").count().orderBy(desc("count")).limit(10)
municipios_con_mas_casos.show()

+------------+-------+
|   municipio|  count|
+------------+-------+
|      BOGOTA|1888137|
|    MEDELLIN| 550790|
|        CALI| 406751|
|BARRANQUILLA| 277989|
|   CARTAGENA| 163526|
| BUCARAMANGA| 142842|
|      IBAGUE|  91598|
| SANTA MARTA|  84863|
|   MANIZALES|  84478|
|      CUCUTA|  77359|
+------------+-------+



In [ ]:
spark.sql("""
          SELECT `municipio` AS municipio, COUNT(*) AS casos_totales
          FROM covid_data
          GROUP BY `municipio`
          ORDER BY casos_totales DESC
          LIMIT 10
          """).show()


+------------+-------------+
|   municipio|casos_totales|
+------------+-------------+
|      BOGOTA|      1888137|
|    MEDELLIN|       550790|
|        CALI|       406751|
|BARRANQUILLA|       277989|
|   CARTAGENA|       163526|
| BUCARAMANGA|       142842|
|      IBAGUE|        91598|
| SANTA MARTA|        84863|
|   MANIZALES|        84478|
|      CUCUTA|        77359|
+------------+-------------+



## 3.3 Los 10 días con más casos de COVID en Colombia

In [ ]:
dias_con_mas_casos = df.groupBy("fecha_reporte").count().orderBy(desc("count")).limit(10)
dias_con_mas_casos.show()

+-------------------+-----+
|      fecha_reporte|count|
+-------------------+-----+
|2022-01-15 00:00:00|35576|
|2022-01-14 00:00:00|34924|
|2021-06-26 00:00:00|33593|
|2021-06-24 00:00:00|32997|
|2021-06-25 00:00:00|32732|
|2021-06-27 00:00:00|32377|
|2022-01-16 00:00:00|32318|
|2022-01-08 00:00:00|31170|
|2022-01-21 00:00:00|31039|
|2022-01-09 00:00:00|30630|
+-------------------+-----+



In [ ]:
spark.sql("""
          SELECT `fecha_reporte` AS fecha_reporte, COUNT(*) AS casos_totales
          FROM covid_data
          GROUP BY `fecha_reporte`
          ORDER BY casos_totales DESC
          LIMIT 10
          """).show()

+-------------------+-------------+
|      fecha_reporte|casos_totales|
+-------------------+-------------+
|2022-01-15 00:00:00|        35576|
|2022-01-14 00:00:00|        34924|
|2021-06-26 00:00:00|        33593|
|2021-06-24 00:00:00|        32997|
|2021-06-25 00:00:00|        32732|
|2021-06-27 00:00:00|        32377|
|2022-01-16 00:00:00|        32318|
|2022-01-08 00:00:00|        31170|
|2022-01-21 00:00:00|        31039|
|2022-01-09 00:00:00|        30630|
+-------------------+-------------+



## 3.4 Distribución de casos por edades

In [ ]:
casos_por_edad = df.groupBy("edad").count().orderBy("edad")
casos_por_edad.show()

+----+-----+
|edad|count|
+----+-----+
|   1|33429|
|   2|26207|
|   3|24223|
|   4|23905|
|   5|24768|
|   6|26380|
|   7|26067|
|   8|28717|
|   9|30256|
|  10|31768|
|  11|34814|
|  12|34208|
|  13|36646|
|  14|39774|
|  15|43189|
|  16|46546|
|  17|52437|
|  18|64310|
|  19|80783|
|  20|93686|
+----+-----+
only showing top 20 rows



In [ ]:
spark.sql("""
          SELECT edad, COUNT(*) AS casos_totales
          FROM covid_data
          GROUP BY edad
          ORDER BY edad
          """).show()

+----+-------------+
|edad|casos_totales|
+----+-------------+
|   1|        33429|
|   2|        26207|
|   3|        24223|
|   4|        23905|
|   5|        24768|
|   6|        26380|
|   7|        26067|
|   8|        28717|
|   9|        30256|
|  10|        31768|
|  11|        34814|
|  12|        34208|
|  13|        36646|
|  14|        39774|
|  15|        43189|
|  16|        46546|
|  17|        52437|
|  18|        64310|
|  19|        80783|
|  20|        93686|
+----+-------------+
only showing top 20 rows



## 3.5 Porcentaje de casos recuperados por departamento

In [ ]:
from pyspark.sql.functions import col

recuperados_por_departamento = df.filter(col("recuperado") == "Recuperado") \
                                  .groupBy("departamento") \
                                  .count() \
                                  .withColumnRenamed("count", "recuperados")

total_por_departamento = df.groupBy("departamento").count().withColumnRenamed("count", "total")

porcentaje_recuperados = recuperados_por_departamento.join(total_por_departamento, "departamento") \
                                                     .withColumn("porcentaje_recuperados",
                                                                 (col("recuperados") / col("total")) * 100) \
                                                     .select("departamento", "porcentaje_recuperados") \
                                                     .orderBy(desc("porcentaje_recuperados"))
porcentaje_recuperados.show()

+--------------+----------------------+
|  departamento|porcentaje_recuperados|
+--------------+----------------------+
|        Tolima|                 100.0|
|  Cundinamarca|                 100.0|
|      Putumayo|                 100.0|
|        VAUPES|     98.37728194726166|
|       GUAINIA|     98.21109123434705|
|     CARTAGENA|     98.15625649743772|
|    SAN ANDRES|     98.13497404345318|
|        BOGOTA|     97.91148629575078|
|       VICHADA|     97.85469107551488|
|     ANTIOQUIA|     97.57932565732655|
|      GUAVIARE|     97.47013352073085|
|  BARRANQUILLA|     97.23550212418476|
|  CUNDINAMARCA|      97.1019916639252|
|STA MARTA D.E.|     97.03757821429834|
|        CALDAS|     97.03488323655117|
|         CHOCO|     96.90590786481198|
|      CASANARE|      96.8939079026373|
|     RISARALDA|     96.89209416247108|
|        BOYACA|      96.8909427832811|
|          META|      96.8889217516683|
+--------------+----------------------+
only showing top 20 rows



In [ ]:
recuperados_df = df.filter(df["recuperado"] == "Recuperado")
recuperados_df.createOrReplaceTempView("recuperados_data")

total_df = df.groupBy("departamento").count().withColumnRenamed("count", "casos_totales")
total_df.createOrReplaceTempView("total_data")

spark.sql("""
          SELECT
              t.`departamento` AS departamento,
              (r.recuperados / t.casos_totales) * 100 AS porcentaje_recuperados
          FROM
              (SELECT `departamento`, COUNT(*) AS Recuperados FROM recuperados_data GROUP BY `departamento`) r
          JOIN
              total_data t
          ON
              r.`departamento` = t.`departamento`
          ORDER BY porcentaje_recuperados DESC
          """).show()

+--------------+----------------------+
|  departamento|porcentaje_recuperados|
+--------------+----------------------+
|        Tolima|                 100.0|
|  Cundinamarca|                 100.0|
|      Putumayo|                 100.0|
|        VAUPES|     98.37728194726166|
|       GUAINIA|     98.21109123434705|
|     CARTAGENA|     98.15625649743772|
|    SAN ANDRES|     98.13497404345318|
|        BOGOTA|     97.91148629575078|
|       VICHADA|     97.85469107551488|
|     ANTIOQUIA|     97.57932565732655|
|      GUAVIARE|     97.47013352073085|
|  BARRANQUILLA|     97.23550212418476|
|  CUNDINAMARCA|      97.1019916639252|
|STA MARTA D.E.|     97.03757821429834|
|        CALDAS|     97.03488323655117|
|         CHOCO|     96.90590786481198|
|      CASANARE|      96.8939079026373|
|     RISARALDA|     96.89209416247108|
|        BOYACA|      96.8909427832811|
|          META|      96.8889217516683|
+--------------+----------------------+
only showing top 20 rows



# 4. Guardar los datos en el bucket de S3 y Google Drive

In [ ]:
departamentos_con_mas_casos.write.csv("s3a://msosavp3/departamentos_mas_casos.csv")
departamentos_con_mas_casos.write.csv(
    "/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/departamentos_mas_casos.csv")

In [ ]:
departamentos_con_mas_casos_df = spark.read.csv("s3a://msosavp3/departamentos_mas_casos.csv", header=True, inferSchema=True)
departamentos_con_mas_casos_df.show()

+------------+-------+
|      BOGOTA|1888137|
+------------+-------+
|   ANTIOQUIA| 955271|
|       VALLE| 572724|
|CUNDINAMARCA| 331331|
|   SANTANDER| 297370|
|BARRANQUILLA| 277989|
|   CARTAGENA| 163526|
|   ATLANTICO| 141072|
|      BOYACA| 131133|
|      TOLIMA| 127764|
+------------+-------+



In [ ]:
municipios_con_mas_casos.write.csv("s3a://msosavp3/municipios_mas_casos.csv")
municipios_con_mas_casos.write.csv(
    "/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/municipios_mas_casos.csv")

In [ ]:
municipios_con_mas_casos_df = spark.read.csv("s3a://msosavp3/municipios_mas_casos.csv", header=True, inferSchema=True)
municipios_con_mas_casos_df.show()

+------------+-------+
|      BOGOTA|1888137|
+------------+-------+
|    MEDELLIN| 550790|
|        CALI| 406751|
|BARRANQUILLA| 277989|
|   CARTAGENA| 163526|
| BUCARAMANGA| 142842|
|      IBAGUE|  91598|
| SANTA MARTA|  84863|
|   MANIZALES|  84478|
|      CUCUTA|  77359|
+------------+-------+



In [ ]:
dias_con_mas_casos.write.csv("s3a://msosavp3/dias_con_mas_casos.csv")
dias_con_mas_casos.write.csv("/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/dias_con_mas_casos.csv")

In [ ]:
dias_con_mas_casos_df = spark.read.csv("s3a://msosavp3/dias_con_mas_casos.csv", header=True, inferSchema=True)
dias_con_mas_casos_df.show()

+------------------------+-----+
|2022-01-15T00:00:00.000Z|35576|
+------------------------+-----+
|     2022-01-14 00:00:00|34924|
|     2021-06-26 00:00:00|33593|
|     2021-06-24 00:00:00|32997|
|     2021-06-25 00:00:00|32732|
|     2021-06-27 00:00:00|32377|
|     2022-01-16 00:00:00|32318|
|     2022-01-08 00:00:00|31170|
|     2022-01-21 00:00:00|31039|
|     2022-01-09 00:00:00|30630|
+------------------------+-----+



In [ ]:
casos_por_edad.write.csv("s3a://msosavp3/casos_por_edad.csv")
casos_por_edad.write.csv("/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/casos_por_edad.csv")

In [ ]:
casos_por_edad_df = spark.read.csv("s3a://msosavp3/casos_por_edad.csv", header=True, inferSchema=True)
casos_por_edad_df.show()

+---+------+
|  1| 33429|
+---+------+
|  2| 26207|
|  3| 24223|
|  4| 23905|
|  5| 24768|
|  6| 26380|
|  7| 26067|
|  8| 28717|
|  9| 30256|
| 10| 31768|
| 11| 34814|
| 12| 34208|
| 13| 36646|
| 14| 39774|
| 15| 43189|
| 16| 46546|
| 17| 52437|
| 18| 64310|
| 19| 80783|
| 20| 93686|
| 21|106309|
+---+------+
only showing top 20 rows



In [ ]:
porcentaje_recuperados.write.csv("s3a://msosavp3/porcentaje_recuperados.csv")
porcentaje_recuperados.write.csv("/content/drive/MyDrive/Tópicos Especiales en Telemática/Proyecto 3/porcentaje_recuperados.csv")

In [ ]:
porcentaje_recuperados_df = spark.read.csv("s3a://msosavp3/porcentaje_recuperados.csv", header=True, inferSchema=True)
porcentaje_recuperados_df.show()

+--------------+-----------------+
|        Tolima|            100.0|
+--------------+-----------------+
|  Cundinamarca|            100.0|
|      Putumayo|            100.0|
|        VAUPES|98.37728194726166|
|       GUAINIA|98.21109123434705|
|     CARTAGENA|98.15625649743772|
|    SAN ANDRES|98.13497404345318|
|        BOGOTA|97.91148629575078|
|       VICHADA|97.85469107551488|
|     ANTIOQUIA|97.57932565732655|
|      GUAVIARE|97.47013352073085|
|  BARRANQUILLA|97.23550212418476|
|  CUNDINAMARCA| 97.1019916639252|
|STA MARTA D.E.|97.03757821429834|
|        CALDAS|97.03488323655117|
|         CHOCO|96.90590786481198|
|      CASANARE| 96.8939079026373|
|     RISARALDA|96.89209416247108|
|        BOYACA| 96.8909427832811|
|          META| 96.8889217516683|
|       BOLIVAR|96.88367711225703|
+--------------+-----------------+
only showing top 20 rows

